In [1]:
from azureml.core import Workspace

ws = Workspace.create(name="myworkspace",
                      subscription_id="b780d7d1-62d8-4fca-aa26-ac7d3521a9ee", 
                      resource_group="seriesly_hot",
                      create_resource_group=False,
                      location='westeurope'
                     )


--------------------------------------------------------------------------- 
 WorkspaceException Traceback (most recent call last)
 <command-3530572963533120> in <module> () 
 5 resource_group = "seriesly_hot" , 
 6 create_resource_group = False , 
 ----> 7 location = 'westeurope' 
 8 )

 /databricks/python/lib/python3.5/site-packages/azureml/core/workspace.py in create (name, auth, subscription_id, resource_group, location, create_resource_group, friendly_name, storage_account, key_vault, app_insights, container_registry, exist_ok) 
 221 key_vault = key_vault , app_insights = app_insights , 
 222 container_registry = container_registry , 
 --> 223 exist_ok=exist_ok)
 224 
 225 @ staticmethod 

 /databricks/python/lib/python3.5/site-packages/azureml/core/workspace.py in _create_legacy (auth, subscription_id, resource_group_name, workspace_name, location, create_resource_group, friendly_name, storage_account, key_vault, app_insights, container_registry, exist_ok) 
 741 app_insights = app_insights , 
 742 containerRegistry = container_registry , 
 --> 743 exist_ok=exist_ok)
 744 if not workspace_object_autorest : 
 745 raise WorkspaceException ( "Couldn't create the workspace." ) 

 /databricks/python/lib/python3.5/site-packages/azureml/_project/_commands.py in create_workspace (auth, resource_group_name, workspace_name, subscription_id, location, create_resource_group, friendly_name, storage_account, key_vault, app_insights, containerRegistry, exist_ok) 
 294 containerRegistry = containerRegistry , 
 295 friendly_name = friendly_name , 
 --> 296 exist_ok=exist_ok)
 297 
 298 

 /databricks/python/lib/python3.5/site-packages/azureml/_workspace/custom.py in ml_workspace_create_resources (auth, client, resource_group_name, workspace_name, location, subscription_id, friendly_name, storage_account, key_vault, app_insights, containerRegistry, exist_ok) 
 52 raise WorkspaceException("Workspace with name '{0}' already exists under"
 53 " resource group with name '{1}'.".format(workspace_name,
 ---> 54 resource_group_name))
 55 except ErrorResponseWrapperException as response_exception : 
 56 if response_exception . response . status_code != 404 : 

 WorkspaceException : Workspace with name 'myworkspace' already exists under resource group with name 'seriesly_hot'.

In [2]:
from azureml.core import Workspace

ws = Workspace.get(name="myworkspace", auth=None, subscription_id="b780d7d1-62d8-4fca-aa26-ac7d3521a9ee", resource_group="seriesly_hot")

In [3]:
from azureml.core.model import Model

model = Model.register(model_path = "/dbfs/FileStore/tables/model.h5",
                       model_name = "model",
                       tags = {"key": "0.1"},
                       description = "test",
                       workspace = ws)

Registering model model

In [4]:
#%%writefile score.py
score = """

# load the model
def init():
    print("This is init()")

# Passes data to the model and returns the prediction
def run(raw_data):
    print("This is run()")
    
""".format(model_name="model")
 
exec(score)
 
with open("score.py", "w") as file:
    file.write(score)

In [5]:
from azureml.core.conda_dependencies import CondaDependencies 

myacienv = CondaDependencies.create(conda_packages=['scikit-learn','numpy','pandas']) #showing how to add libs as an eg. - not needed for this model.

with open("mydeployenv.yml","w") as f:
    f.write(myacienv.serialize_to_string())


In [6]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'name':'Databricks Azure ML ACI'}, 
                                               description = 'Time series')

In [7]:
from azureml.core.image import ContainerImage

# Image configuration
image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                 runtime = "python",
                                                 conda_file = "mydeployenv.yml",
                                                 description = "Time series",
                                                 tags = {"data": "series", "type": "series"}
                                                 )

In [8]:
# Register the image from the image configuration
image = ContainerImage.create(name = "myimage", 
                              models = [model], #this is the model object
                              image_config = image_config,
                              workspace = ws
                              )

Creating image

In [9]:
from azureml.core.webservice import Webservice

service_name = 'hot-service'

myservice = Webservice(workspace=ws, name=service_name).delete()

# Webservice creation
myservice = Webservice.deploy_from_model(
      workspace=ws, 
      name=service_name,
      deployment_config = aciconfig,
      models = [model],
      image_config = image_config
    )

myservice.wait_for_deployment(show_output=True)

Creating image
Image creation operation finished for image hot-service:3, operation "Succeeded"
Creating service
Running.....................
SucceededACI service creation operation finished, operation "Succeeded"